In [9]:
#Imports
import pykep as pk
import pygmo as pg
import pygmo_plugins_nonfree as ppnf
algo = pg.algorithm(ppnf.snopt7(True))  # True enables verbose
from pykep.planet import jpl_lp
from pykep import epoch
import numpy as np
import matplotlib.pyplot as plt
import math

# Load Spice Kernel
pk.util.load_spice_kernel("/home/m/mmk40/Desktop/Planet Position/de431.bsp")

In [26]:
#Define Planetary Ephimeris
earth = pk.planet.spice('EARTH', 'SUN', 'ECLIPJ2000', 'NONE')                    # position of Earth using spice kernel
venus = pk.planet.spice('VENUS', 'SUN', 'ECLIPJ2000', 'NONE')                    # position of Venus using spice kernel
jupiter = pk.planet.spice("JUPITER BARYCENTER", 'SUN', 'ECLIPJ2000', 'NONE')     # position of Jupiter using spice kernel
mercury = pk.planet.spice("MERCURY", 'SUN', 'ECLIPJ2000', 'NONE')                # position of Mercury using spice kernel
#print(earth, venus, jupiter)                                                    # print checks

# Define Planetary safe radius or else it gives an error in the problem if using spice

earth.radius = 6371.0          # Earth's mean radius and safe radius (Earth radius + atmosphere ~ 6371 km + 200 km)
earth.safe_radius = 6571.0     # 200 km safety margin

venus.radius = 6052.0          # Venus radius and safe radius (Venus mean radius ~ 6052 km)
venus.safe_radius = 10         # 10 times Venus radius

jupiter.radius = 69911.0       # Jupiter radius and safe radius (mean radius ~ 69911 km)
jupiter.safe_radius = 30       # 20 times Jupiter Radius

#print("Venus radius:", venus.radius, type(venus.radius))                    # print checks
#print("Venus safe radius:", venus.safe_radius, type(venus.safe_radius))     # print checks
#print("Normalized safe radius:", venus.safe_radius / venus.radius)          # print checks



# Define Launch and Arrival Dates
departure = int(pk.epoch_from_string("1850-Jan-01 00:00:00").mjd2000)  # ≈ 12784; Turns the date into mjd
arrival = int(pk.epoch_from_string("2040-Jan-01 00:00:00").mjd2000)    # ≈ 18262; Turns the date into mjd
#print(departure, arrival)                                              # print checks

In [47]:
# Define the problem i.e. Earth-Venus-Jupiter flyby with 1 DSM.
udp = pk.trajopt.mga_1dsm(
    seq=[jpl_lp('earth'), jpl_lp('venus'), jpl_lp('earth'), jpl_lp('jupiter')],
    t0=[epoch(departure),epoch(arrival)],             # Departure window (in MJD2000)
    tof=[[250,480], [300,400], [300,900]],       # TOF for Earth→Venus, Venus→Jupiter
    vinf=[6.0,10.0],                      # Departure hyperbolic excess speed [km/s]
    add_vinf_dep=False,                   # Objective includes v-infinity at Earth/Venus
    add_vinf_arr=False                   # # Objective includes v-infinity at Venus/Jupiter
)

In [48]:
# Create Pygmo set and tolerences
prob = pg.problem(udp)
prob.c_tol = 1e-4
#print(prob)   # print check
#print("Lower bounds:", prob.get_lb())
#print("Upper bounds:", prob.get_ub())

In [55]:
# --- Global search with SADE ---
uda = pg.sade(gen=8000)  # increase generations for better coverage
archi = pg.archipelago(algo=uda, prob=prob, n=30, pop_size=60)

print("Running SADE global search ...")
archi.evolve(25)
archi.wait()

# Extract champion
champ_fs = archi.get_champions_f()
champ_xs = archi.get_champions_x()
idx = np.argmin([f[0] if hasattr(f,'__len__') else f for f in champ_fs])
best_x = champ_xs[idx]
print("Global best objective:", champ_fs[idx])
#udp.pretty(archi.get_champions_x()[idx])

# --- Local refinement with NLopt BOBYQA ---
print("\n\nPolishing best candidate with BOBYQA (open-source local solver) ...")
local_algo = pg.algorithm(pg.nlopt('bobyqa'))
local_prob = pg.problem(udp)
pop = pg.population(local_prob, size=1)
pop.set_x(0, best_x)
pop = local_algo.evolve(pop)

# Final solution
final_x = pop.get_x()[0]
final_f = pop.get_f()[0]
print("Polished objective:", final_f)

udp.pretty(final_x)

Running SADE global search ...
Global best objective: [1243.32729534]


Polishing best candidate with BOBYQA (open-source local solver) ...
Polished objective: [1243.32729533]
First Leg: earth to venus
Departure: 1983-Feb-13 00:59:24.641437 (-6165.958742575955 mjd2000) 
Duration: 349.6920147512976days
VINF: 6.000000000329764 km/sec
DSM after 133.7879645646639 days
DSM magnitude: 6.305761104561466e-08m/s

leg no. 2: venus to earth
Duration: 392.23212605222767days
Fly-by epoch: 1984-Jan-28 17:35:54.715950 (-5816.266727824657 mjd2000) 
Fly-by radius: 2.1612077373497716 planetary radii
DSM after 192.84787906682675 days
DSM magnitude: 4.3665021400897826e-08m/s

leg no. 3: earth to jupiter
Duration: 886.7326027818042days
Fly-by epoch: 1985-Feb-23 23:10:10.406862 (-5424.03460177243 mjd2000) 
Fly-by radius: 1.1000000000000005 planetary radii
DSM after 88.67326027818042 days
DSM magnitude: 1243.3272952271614m/s

Arrival at jupiter
Arrival epoch: 1987-Jul-30 16:45:07.287210 (-4537.301998990625 m

In [ ]:
# Choose optimization algorithm
uda = pg.sade(gen=6000)

# Initialize population and optimize
archi = pg.archipelago(algo=uda, prob=udp, n=20, pop_size=40)
print("Running a Self-Adaptive Differential Evolution Algorithm .... on 10 parallel islands")
archi.evolve(15)
archi.wait()
sols = archi.get_champions_f()
idx = sols.index(min(sols))
print("Done!! Solutions found are: ", archi.get_champions_f())
udp.pretty(archi.get_champions_x()[idx])